In [ ]:
import tensorflow as tf
import cv2
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os
import threading
from concurrent.futures import ThreadPoolExecutor
import time
import multiprocessing
from tensorflow import keras
import math

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(28, 28, 1)),
    
    # First Convolutional Block
    tf.keras.layers.Conv2D(64, (3, 3), kernel_initializer='he_normal', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),
    
    # Second Convolutional Block
    tf.keras.layers.Conv2D(128, (3, 3), kernel_initializer='he_normal', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),
    
    # Third Convolutional Block (added for depth)
    tf.keras.layers.Conv2D(256, (3, 3), kernel_initializer='he_normal', padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.ReLU(),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),

    # Flatten and Fully Connected Layers
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, kernel_regularizer= tf.keras.regularizers.l2(l= 0.016), 
                          activity_regularizer= tf.keras.regularizers.l1(0.006),
                          bias_regularizer= tf.keras.regularizers.l1(0.006), 
                          activation= 'relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(11, activation='softmax')
])

model.load_weights('sudoku2.h5')

In [ ]:
puzzles = [os.path.join('puzzles', img) for img in os.listdir('./puzzles/')]

In [ ]:
import numpy as np
import tensorflow as tf
from concurrent.futures import ThreadPoolExecutor

class SudokuDigitPredictor:

    def __init__(self, model, confidence=0.9):
        self.model = model
        self.confidence = confidence
        self.classes = [str(i) for i in range(11)]
        
    # def preprocess_squares_in_parallel(self, warped_sudoku):
    #     with ThreadPoolExecutor(max_workers=4) as executor:  # adjust max_workers as needed
    #         squares = list(executor.map(self.preprocess_for_prediction, [warped_sudoku[y_start:y_start + 28, x_start:x_start + 28] for i in range(9) for j in range(9, x_start=i*28, y_start=j*28)]))
    #     return squares

    def preprocess_for_prediction(self, square):
        """
        Preprocess the cropped square for prediction.
        This depends on how your model expects the input. 
        For example, rescaling, resizing, or reshaping might be necessary.
        """
        # Ensure the square is in grayscale
        if len(square.shape) == 3 and square.shape[2] == 3:
            square = cv2.cvtColor(square, cv2.COLOR_BGR2GRAY)
            
        # Remove 10 pixels from all sides
        square = square[3:-3, 3:-3]
        # Apply binary threshold
        _, binary = cv2.threshold(square, 127, 255, cv2.THRESH_BINARY)

        # Enhance contrast using histogram equalization
        enhanced_binary = cv2.equalizeHist(binary)
        
        processed_square = cv2.resize(enhanced_binary, (28, 28))  # Resizing
        processed_square = np.expand_dims(processed_square, axis=-1)  # Ensure it has a single channel
        return processed_square

    def predict_digits(self, warped_sudoku):
        """
        Predicts digits for a given warped sudoku grid.
        """
        squares = []
        
        # Crop the sudoku grid into 81 squares
        for i in range(9):
            for j in range(9):
                x_start, y_start = i * 28, j * 28
                square = warped_sudoku[x_start:x_start + 28, y_start:y_start + 28]
                squares.append(self.preprocess_for_prediction(square))

        # Convert list to numpy array for batch prediction
        squares_np = np.stack(squares)

        # Predict
        predictions = self.model.predict_on_batch(squares_np)

        max_confidences = np.amax(predictions, axis=1)
        class_indices = np.argmax(predictions, axis=1)
        
        # Ensure self.classes is a numpy array
        class_array = np.array(self.classes)

        # Determine where predictions exceed the confidence threshold
        above_confidence = max_confidences > self.confidence

        # Use boolean indexing to get relevant classes
        relevant_classes = class_array[class_indices]

        # Where predictions are above the confidence threshold, use predicted class, else '10'
        predicted_classes = np.where(above_confidence, relevant_classes, '10').tolist()
        
        # Convert cell_values to integer-based numpy array, where '10' becomes 0
        cell_values_int = np.array([0 if value == '10' else int(value) for value in predicted_classes])

        # Reshape the numpy array to the desired 9x9 shape
        grid = cell_values_int.reshape(9, 9)
        
        return grid

In [ ]:
class SudokuGridHighlighter:

    def __init__(self):
        self.last_contour = None
        self.consistent_detections = 0
        self.detection_threshold = 5
        self.max_area_detected = 0
        self.digit_predictor = SudokuDigitPredictor(model)
        self.grid_prediction = None
        self.sudoku_solution = None
        self.frame_count = 0
        self.predict_every_n_frames = 10
        self.prev_frame = None

    def frame_difference(self, current_frame, prev_frame):
        """
        Compute the difference between current frame and previous frame.
        This will be a simple method, but can be made more sophisticated.
        """
        diff = cv2.absdiff(current_frame, prev_frame)
        return np.average(diff)
    
    def preprocess(self, image):
        """Preprocess the image for contour detection."""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        blur = cv2.GaussianBlur(gray, (3, 3), 6)
        return cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 2)

    def main_outline(self, contours):
        """Get the largest 4-sided contour in the image."""
        max_area = 0
        biggest_contour = np.array([])
        
        for contour in contours:
            area = cv2.contourArea(contour)
            
            # Adjust the area threshold dynamically
            if area > self.max_area_detected * 0.5 and area > max_area:
                peri = cv2.arcLength(contour, True)
                approx = cv2.approxPolyDP(contour, 0.02 * peri, True)
                
                if len(approx) == 4:
                    biggest_contour = approx
                    max_area = area
                        
        return biggest_contour, max_area

    def get_cropped_region(self, frame, padding=40):
        """Crops the frame based on the last detected contour and returns the cropped image and its offset."""
        if self.last_contour is None:
            return frame, (0, 0)

        x, y, w, h = cv2.boundingRect(self.last_contour)
        x = max(0, x - padding)
        y = max(0, y - padding)
        w += 2 * padding
        h += 2 * padding

        return frame[y:y+h, x:x+w], (x, y)

    def order_points(self, pts):
        """
        Order the 4 points in this format: top-left, top-right, bottom-right, bottom-left
        """
        rect = np.zeros((4, 2), dtype="float32")

        s = pts.sum(axis=1)
        rect[0] = pts[np.argmin(s)]  # top-left will have the smallest sum
        rect[2] = pts[np.argmax(s)]  # bottom-right will have the largest sum

        diff = np.diff(pts, axis=1)
        rect[1] = pts[np.argmin(diff)]  # top-right will have the smallest difference
        rect[3] = pts[np.argmax(diff)]  # bottom-left will have the largest difference

        return rect

    def warp_perspective(self, image, contour):
        """Warp the perspective to get a top-down view of the image."""
        
        if contour.shape[0] != 4 or contour is None:
            return (None, None)

        ordered_contour = self.order_points(contour.reshape(4, 2))
        src = np.array([
            [0, 0],
            [252 - 1, 0],
            [252 - 1, 252 - 1],
            [0, 252 - 1]
        ], dtype="float32")

        matrix = cv2.getPerspectiveTransform(ordered_contour, src)
        warped = cv2.warpPerspective(image, matrix, (252, 252))
        
        
        if warped is None or matrix is None:
            return (None, None)
        return (warped, matrix)

    
    def check_sudoku_squares(self, warped_image):
        count_valid_squares = 0

        for i in range(9):
            for j in range(9):
                x_start, y_start = i * 28, j * 28
                square = warped_image[y_start:y_start + 28, x_start:x_start + 28]
                edges = cv2.Canny(square, 50, 150, apertureSize=3)
                lines = cv2.HoughLines(edges, 1, np.pi / 180, 15)
                
                if lines is not None:
                    count_valid_squares += 1

        return count_valid_squares
    
    def _solve_sudoku(self, grid):
        def is_valid_move(x, y, val, grid):
            """Check if placing val at grid[x][y] is valid."""
            
            # Check row and column
            for i in range(9):
                if grid[x][i] == val or grid[i][y] == val:
                    return False
                    
            # Check 3x3 box
            startRow, startCol = 3 * (x // 3), 3 * (y // 3)
            for i in range(3):
                for j in range(3):
                    if grid[i + startRow][j + startCol] == val:
                        return False
                        
            return True

        def _solve_sudoku_helper(grid_copy):
            """Helper function for solve_sudoku."""
            
            for x in range(9):
                for y in range(9):
                    if grid_copy[x][y] == 0:
                        for val in range(1, 10):
                            if is_valid_move(x, y, val, grid_copy):
                                grid_copy[x][y] = val
                                if _solve_sudoku_helper(grid_copy):
                                    return True
                                grid_copy[x][y] = 0
                        return False
            return True

        """Fill the grid using backtracking and return a solved copy."""
            
        # Create a deep copy of the original grid
        grid_copy = np.copy(grid)
        
        if _solve_sudoku_helper(grid_copy):
            return grid_copy
        return None  # Return None if the grid couldn't be solved
    
    def solve_sudoku_timed(self, grid, timeout=1):
        """
        Attempt to solve the Sudoku with a time limit.
        If the Sudoku is solved within the time limit, return the solution.
        If not, return None.
        """
        print('solving...')
        result = {"solution": None}

        # Terminate the previous thread if it's still running
        if hasattr(self, "current_thread") and self.current_thread.is_alive():
            # Termination of threads in Python is tricky, but setting a flag to check in the thread's function might work.
            # Here's a simple solution that assumes the worker thread periodically checks a flag
            self.terminate_thread = True
            self.current_thread.join()  # Wait for it to finish (could also be timed)
            self.terminate_thread = False

        def worker():
            if not hasattr(self, "terminate_thread") or not self.terminate_thread:
                solution = self._solve_sudoku(grid)
                result["solution"] = solution

        t = threading.Thread(target=worker)
        t.start()
        t.join(timeout=timeout)  # half a second time limit

        if t.is_alive():  # if thread is still running after 0.5 seconds
            print("couldn't solve in time")
            return None  # or an indication of failure if you prefer

        self.current_thread = t  # Save this thread as the current thread
        return result["solution"]

       
    def detect_and_highlight_grid(self, image):
        """Detects the Sudoku grid and highlights it."""
        

        _original_preprocessed_image = self.preprocess(image)
        _original_contours, _ = cv2.findContours(_original_preprocessed_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        _original_biggest_contour, _original_max_area = self.main_outline(_original_contours)
        _original_warped, _original_matrix = self.warp_perspective(image, _original_biggest_contour)

        cropped_image, (offset_x, offset_y) = self.get_cropped_region(image)
        preprocessed_image = self.preprocess(cropped_image)
        contours, _ = cv2.findContours(preprocessed_image, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        biggest_contour, max_area = self.main_outline(contours)
        warped, matrix = self.warp_perspective(cropped_image, biggest_contour)
        
        if warped is None:
            self.consistent_detections -= 1
            if self.consistent_detections < -3:
                self.last_contour = None
                self.consistent_detections = 0
            return image

        valid_squares = self.check_sudoku_squares(warped)
        
        if valid_squares > 81 * 0.75:
            if biggest_contour.size != 0:
                biggest_contour += np.array([offset_x, offset_y]).reshape(1, 1, 2)

            if max_area > self.max_area_detected:
                self.max_area_detected = max_area

            cv2.drawContours(image, [biggest_contour], 0, (255, 105, 180), 6)
            self.consistent_detections += 1

            if self.consistent_detections > self.detection_threshold:
                self.last_contour = biggest_contour

            # Resizing the warped Sudoku grid to display on the main image
            grid_display_size = (100, 100)  # Adjust size as per your requirements
            small_grid = cv2.resize(warped, grid_display_size)

            # Positioning the warped grid on the bottom right corner of the main image
            image[-grid_display_size[1]:, -grid_display_size[0]:] = small_grid
            
            # Reducing number of predictions
            if self.frame_count % self.predict_every_n_frames == 0:
                if self.prev_frame is not None:
                    diff = self.frame_difference(image, self.prev_frame)
                    
                    if diff > 12:
                        print(f"Grid shift detected [{diff}]")
                        self.grid_prediction = self.digit_predictor.predict_digits(warped)
                        # Use the timed solver here
                        solved_grid = self.solve_sudoku_timed(self.grid_prediction)
                        if solved_grid is not None:
                            self.sudoku_solution = solved_grid
                else:
                    self.grid_prediction = self.digit_predictor.predict_digits(warped)
                    # And here
                    solved_grid = self.solve_sudoku_timed(self.grid_prediction)
                    if solved_grid is not None:
                        self.sudoku_solution = solved_grid

        else:
            self.consistent_detections -= 1
            if self.consistent_detections < -3:
                self.last_contour = None
                self.consistent_detections = 0
        
        self.frame_count += 1
        self.prev_frame = image.copy()
        
        if self.sudoku_solution is not None:
            return self.overlay(image, _original_warped, _original_matrix)
        
        return image
    
    def overlay_solution_on_image(self, image, grid, original_grid):
        """
        Overlay the solved numbers on the original Sudoku image.

        Parameters:
        - image: The original Sudoku image.
        - grid: The solved Sudoku grid.
        - original_grid: The transcribed Sudoku grid from the image.
        """
        
        # Convert grayscale to BGR if image has only one channel
        if len(image.shape) == 2 or image.shape[2] == 1:
            image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)

        # Dimensions of the image
        height, width = image.shape[0:2]

        # Dimensions of each cell
        cell_width = width // 9
        cell_height = height // 9

        # Set appropriate font scale and thickness considering the size of the cells
        font_scale = min(cell_width, cell_height) / 40  # Adjust the denominator as needed
        thickness = math.ceil(font_scale) * 2

        for x in range(9):
            for y in range(9):
                if original_grid[x][y] == 0:  # If the cell was originally empty
                    cell_value = str(grid[x][y])

                    # Calculate text size
                    (text_width, text_height), _ = cv2.getTextSize(cell_value, cv2.FONT_HERSHEY_SIMPLEX, font_scale, thickness)

                    # Determine the position to put the text (approximately in the center of the cell)
                    pos = (y * cell_width + (cell_width - text_width) // 2, 
                        x * cell_height + (cell_height + text_height) // 2)

                    # Overlay the text on the copied image
                    cv2.putText(image, cell_value, pos, cv2.FONT_HERSHEY_SIMPLEX, font_scale, (255, 105, 180), thickness)
        return image    
    

    def inverse_transform_and_overlay(self, overlayed_image, original_image, matrix):
        """Inverse transform the overlayed_image and overlay on original image."""
        inversed = cv2.warpPerspective(overlayed_image, np.linalg.inv(matrix), (original_image.shape[1], original_image.shape[0]))
        mask = cv2.warpPerspective(np.ones_like(overlayed_image) * 255, np.linalg.inv(matrix), (original_image.shape[1], original_image.shape[0]))
        
        # Apply mask to original image to black out the Sudoku area
        masked_original = cv2.bitwise_and(original_image, 255 - mask)

        # Combine the masked original image with the inversed transformed image
        combined = cv2.bitwise_or(masked_original, inversed)

        return combined


    def overlay(self, image, warped_image, matrix):        
        overlayed_image = self.overlay_solution_on_image(warped_image, self.sudoku_solution, self.grid_prediction)
        combined_image = self.inverse_transform_and_overlay(overlayed_image, image, matrix)
        return combined_image

In [ ]:
highlighter = SudokuGridHighlighter()

puzzles = [os.path.join('puzzles', img) for img in os.listdir('./puzzles/')]

puzzle = cv2.imread(puzzles[0])

highlighted_frame = highlighter.detect_and_highlight_grid(puzzle)
fig, ax = plt.subplots(1, 2, figsize=(10, 5))
ax[0].imshow(puzzle)
ax[1].imshow(highlighted_frame)

In [ ]:
def webcam_feed():
    highlighter = SudokuGridHighlighter()
    video_capture = cv2.VideoCapture(0)
    video_capture.set(cv2.CAP_PROP_FPS, 20)
    
    # Set resolution
    video_capture.set(cv2.CAP_PROP_FRAME_WIDTH, 128)
    video_capture.set(cv2.CAP_PROP_FRAME_HEIGHT, 128)

    if not video_capture.isOpened():
        print("Error: Could not open webcam.")
        return

    while True:
        ret, frame = video_capture.read()

        if not ret:
            print("Failed to grab frame.")
            break

        highlighted_frame = highlighter.detect_and_highlight_grid(frame)
        cv2.imshow('Webcam Feed - Sudoku Grid Highlighter', highlighted_frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    video_capture.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    webcam_feed()
